In [1]:
import numpy as np

In [2]:
from lib.noglobal import noglobal
from lib.math.coodinate import convert_3d_to_2d





@noglobal()
def convert(points:np.array, from_point:np.array,to_point:np.array):
    """
    from_point: 変更前の原点
    to_point: 変更後の原点
    """
    
    #if(type(points)):
        
    
    
    
    
    #print(type(points))
    
    a = np.linalg.norm(to_point[:2]-from_point[:2],ord=2)
    b = np.linalg.norm(to_point-from_point,ord=2)
    
    if(a<1e-6):
        sin_theta = 0
        cos_theta = 1
    else:
        tmp_a = (to_point - from_point)/a
        sin_theta = tmp_a[1]
        cos_theta = tmp_a[0];
                    
    if (b<1e-6):
        sin_phi = 0;                
        cos_phi = 1;                
    else:
        tmp_b = (to_point - from_point)/b
        sin_phi = tmp_b[2];        
        cos_phi = a/b
    
    rotmat = [
        [cos_phi*cos_theta, cos_phi*sin_theta, sin_phi],
        [-1*sin_theta, cos_theta,0],
        [-1*sin_phi*cos_theta, -sin_phi*sin_theta, cos_phi]
    ]
                    
    """
    奥行き: depth
    水平: horizon
    垂直: vertical        
    """
    ret_df = (rotmat@(points-from_point).T).T
    ret_df = ret_df.rename(columns = {0:"depth",1:"horizon",2:"vertical"})
    
    
    return ret_df;

#convert(df.loc[0,["x","y","z"]], frompoint,topoint)
frompoint = np.array([0,1,0])
topoint = np.array([0,0,0])

#convert(df, frompoint,topoint)
#convert_3d_to_2d(df, frompoint,topoint)

In [3]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(data = {
    "x":[0,3,4,5,6],
    "y":[0,3,4,5,6],
    "z":[0.5,0.5,0.5,0.5,0.5]
});


original_fig = px.scatter_3d(df,x="x",y = "y",z = "z");




from lib.math.coodinate import generate_xz_rotation_maxrix
import numpy as np

toPoint = np.array([60,27,5.0])
from_base_point = np.array([150,27,5.0])


length_vector = from_base_point - toPoint

rotation_matrix = generate_xz_rotation_maxrix(-1*90,isDeg=True);
rotated_point = toPoint + rotation_matrix@length_vector

rotated_point

In [4]:
from external_lib.NFLlib.features import add_track_features
base_dir = "/work/data/input/nfl-health-and-safety-helmet-assignment"

debug = True
n_debug_samples = 1
random_state = 42

labels = pd.read_csv(f"{base_dir}/train_labels.csv")

if debug:
    tracking = pd.read_csv(f'{base_dir}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{base_dir}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/test_baseline_helmets.csv')
    
def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if 'video' not in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df

tracking = add_track_features(tracking)

if debug:
    helmets = add_cols(helmets)
    labels = add_cols(labels)
    
    # Select `n_debug_samples` worth of videos to debug with
    sample_videos = labels['video'].drop_duplicates() \
        .sample(n_debug_samples, random_state=random_state).tolist()
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]
tracking.shape, helmets.shape, labels.shape

((6424, 18), (8746, 8), (7888, 15))

### Endzone

In [5]:
extract_tracking = tracking[tracking["game_play"]=="57783_003374"]

extract_tracking["z"]  = 5.0

In [6]:
#x = tracking_df['x']
#y = tracking_df['y']
#min_dist = 10000
#min_idx = None
#min_x = None
        
#np.array((x,y))
    
    

In [7]:
#toPoint = np.array([60,27,5.0])
#from_base_point = np.array([60,27,150])


#from lib.math.coodinate import convert_3d_to_2d

#convert_3d_to_2d(extract_tracking[["x","y","z"]],from_point=from_base_point,to_point=toPoint)


In [8]:
from lib.math.coodinate import generate_rotation_maxrix
    
@noglobal(excepts = ["rotate_arr"])
def dist_rot_v2(tracking_df, a2,view,DIG_MAX,DIG_STEP,max_iter):
    tracking_df = tracking_df.sort_values('x')
    x = tracking_df['x']
    y = tracking_df['y']
    min_dist = 10000
    min_idx = None
    min_x = None
    #display(tracking_df[["x","y"]])
    #for dig in range(-1*DIG_MAX,DIG_MAX+1,DIG_STEP):
    for dig in range(-1*DIG_MAX,DIG_MAX+1,DIG_STEP):
        
        ## コートの４角の一つを中心としたときの座標変換
        ## x: コートの長い方に沿ったときの位置
        ## y: コートの短い方に沿ったときの位置
        ## arr: 座標変換後のプレイヤーの位置
        
        if (dig <0):
            arr = rotate_arr_v2(tracking_df,view,180 - np.abs(dig));
        else:
            arr = rotate_arr_v2(tracking_df,view,dig);
        
    
        
        arr = arr[["vertical","horizon"]].values.T
        
        
        
        
        ##  a2: ヘルメットの座標        
        this_dist, this_idx = dist_for_different_len(np.sort(arr[0]), a2,max_iter)
        if min_dist > this_dist:
            min_dist = this_dist
            min_idx = this_idx
            min_x = arr[0]
        
            
            
    
    tracking_df['x_rot'] = min_x
    player_arr = tracking_df.sort_values('x_rot')['player'].values
    players = np.delete(player_arr,min_idx)
    
    
    return min_dist, players        
    

@noglobal()
def mapping_df_v2(args):
    video_frame, df = args[:2]
    
    parameter = args[2]
    
    tracking = parameter["tracking"]
    CONF_THRE = parameter["CONF_THRE"];
    DIG_STEP = parameter["DIG_STEP"];
    DIG_MAX = parameter["DIG_MAX"];
    max_iter = parameter["max_iter"];
    
    gameKey,playID,view,frame = video_frame.split('_')
    
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
    
    
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame]    
    
    len_this_tracking = len(this_tracking)
    
    df['center_h_p'] = (df['left']+df['width']/2).astype(int)
    df['center_h_m'] = (df['left']+df['width']/2).astype(int)*-1
    df = df[df['conf']>CONF_THRE].copy()
    if len(df) > len_this_tracking:
        df = df.tail(len_this_tracking)
    df_p = df.sort_values('center_h_p').copy()
    df_m = df.sort_values('center_h_m').copy()
    
    if view == 'Endzone':
        this_tracking['x'], this_tracking['y'] = this_tracking['y'].copy(), this_tracking['x'].copy()
    a2_p = df_p['center_h_p'].values
    a2_m = df_m['center_h_m'].values

    #min_dist_p, min_detete_idx_p = dist_rot(this_tracking ,a2_p,DIG_MAX,DIG_STEP,max_iter)
    #min_dist_m, min_detete_idx_m = dist_rot(this_tracking ,a2_m,DIG_MAX,DIG_STEP,max_iter)
    min_dist_p, min_detete_idx_p = dist_rot_v2(this_tracking ,a2_p,view,DIG_MAX,DIG_STEP,max_iter)
    min_dist_m, min_detete_idx_m = dist_rot_v2(this_tracking ,a2_m,view,DIG_MAX,DIG_STEP,max_iter)
    
    
    if min_dist_p < min_dist_m:
        min_dist = min_dist_p
        min_detete_idx = min_detete_idx_p
        tgt_df = df_p
    else:
        min_dist = min_dist_m
        min_detete_idx = min_detete_idx_m
        tgt_df = df_m
    #print(video_frame, len(this_tracking), len(df), len(df[df['conf']>CONF_THRE]), this_tracking['x'].mean(), min_dist_p, min_dist_m, min_dist)
    tgt_df['label'] = min_detete_idx
    return tgt_df[['video_frame','left','width','top','height','label']]
            
 

In [9]:
import pandas as pd

view="Endzone"
tracking["z"] = 5.0
ext_tracking = tracking[tracking["game_play"] == "57783_003374"]
ext_tracking = ext_tracking.sort_values('x')
ext_tracking = ext_tracking.reset_index();

In [10]:
view="Sideline"
arr = mapping_df_v2(ext_tracking,view);


#ext_tracking[["horizon","vertical","depth"]] = arr[["horizon","vertical","depth"]]

TypeError: mapping_df_v2() takes 1 positional argument but 2 were given

In [ ]:
display(ext_tracking[["horizon","vertical","depth","x","y","z"]])

In [ ]:
#helmets
helmets[helmets["video_frame"].str.contains("57783_003374")]

In [ ]:
#ext_tracking[["x","y","rot_x","rot_y"]]

#ext_tracking["rot_x"].plot()